#  <center> Taller  de Aprendizaje Automático </center>
##  <center> Taller 4: Detección de Anomalías  </center>

# Introducción

En la siguiente actividad se trabajará en la detección de anomalías sobre redes de computadoras a partir de datos de tráfico. Para esto se utilizará una parte del conjunto [KDD Cup'99](https://scikit-learn.org/stable/datasets/real_world.html#kddcup99-dataset) pensada para evaluar métodos de detección de anomalías. 

Para los problemas de detección de anomalías generalmente no se cuenta con datos etiquetados para entrenar un detector. Por su definición las anomalías son eventos raros y por lo tanto poco frecuentes, lo que dificulta el etiquetado. Es por esto que este tipo de tareas generalmente son no supervisadas.

El enfoque más habitual para implementar soluciones para este tipo de problemas, es crear un modelo base a partir de un conjunto de datos "normales", es decir de los cuales se tenga cierta certeza de que todos fueron adquiridos en una situación normal. Luego en producción se detectarán como datos anómalos todos aquellos que no se ajusten a este modelo. Para saber el grado de ajuste de los datos se debe seleccionar un punto de operación, es decir, determinar cuándo un dato se considera anómalo. En un ejemplo real, el cliente primero debería proporcionar una cantidad considerable de datos que representen el comportamiento normal de su sistema. Luego que se tiene el mejor modelo posible de estos datos, junto con el cliente, que es el que conoce su sistema, se debe determinar el punto de operación a partir del compromiso entre detectar la mayor cantidad de anomalías y obtener la menor cantidad de falsas alarmas posibles.

Para hacer investigación en la detección de anomalías, existen conjuntos de datos como el que se trabajará en esta actividad que si tienen etiquetas. Generalmente estas se obtienen provocando fallas y/o ataques intencionales a un sistema que se encuentra funcionando de manera normal. En esta actividad se separará el conjunto de entrenamiento en dos partes. La primera con una gran proporción de datos etiquetados como normales, simulará ser el conjunto que el cliente nos proporciona para entrenar nuestro modelo. El otro conjunto tendrá datos etiquetados como normales o como anómalos, que se utilizará para definir el punto de operación. Luego, se descargarán los datos de test asociados a este problema para evaluar la puesta en producción del modelo. 

## Objetivos


*   Abordar un problema de detección de anomalías, y ver las diferencias con un problema de clasificación convencional.
*   Trabajar con algoritmos de aprendizaje no supervisado.
*   Crear detectores compatibles con los *pipelines* de *scikit-learn*.


# Formas de trabajo

### Opción 1: Trabajar localmente

Descargar los datos en su máquina personal y trabajar en su propio ambiente de desarrollo.

`conda activate TAA-py38`              
`jupyter-notebook`    

Los paquetes faltantes se pueden instalar desde el notebook haciendo:     
` !pip install paquete_faltante` 

### Opción 2:  Trabajar en *Colab*. 

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/TAA-fing/TAA-2022/blob/main/talleres/taller4_anomalias.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Ejecutar en Google Colab</a>
  </td>
</table>

Se puede trabajar en Google Colab. Para ello es necesario contar con una cuenta de **google drive** y ejecutar un notebook almacenado en dicha cuenta. De lo contrario, no se conservarán los cambios realizados en la sesión. En caso de ya contar con una cuenta, se puede abrir el notebook y luego ir a `Archivo-->Guardar una copia en drive`. 

# Datos

### Parte 1 - Levantar los Datos

#### Conjunto de Entrenamiento

El subconjunto de [KDD Cup'99](https://scikit-learn.org/stable/datasets/real_world.html#kddcup99-dataset) con el que se trabajará contiene 100655 instancias donde cada una cuenta con 41 características. Por más información sobre el contenido de las características haga clic [aquí](http://kdd.ics.uci.edu/databases/kddcup99/task.html). Además, se cuenta con la columna *'labels'* que indica si el dato es normal o, de no serlo, indica el tipo de anomalía.

#### Objetivos:

 - Levantar el conjunto de datos de entrenamiento. 
 - Determinar la relación entre datos normales y anómalos.  

In [1]:
!pip install -U scikit-learn

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_kddcup99

#Se obtienen los datos en formato de diccionario
KDDSA = fetch_kddcup99(subset='SA', as_frame=True, )

#A partir del diccionario se crea un Dataframe con los datos
df = pd.DataFrame(data=KDDSA.frame.values, columns=KDDSA.frame.columns)

#Se estandariza el formato de los datos en el Dataframe 
types = [float, str, str,str, float, float, str, float, float, float, float, str, float, float,float, float, float, float, float, float, str, str, 
         float, float, float, float,float, float, float, float,float, float, float, float, float, float, float,float, float, float, float, str]

columns = df.columns
for i in range(len(columns)):
    df[columns[i]] = df[columns[i]].astype(types[i])
    if types[i] == str:
        df[columns[i]]= df[columns[i]].str.replace("b'", "")
        df[columns[i]]= df[columns[i]].str.replace("'", "")

#Visualizo
df.head()

#### Conjunto de Test

Este problema tiene disponible un conjunto de datos para Test, estos se pueden encontar [aquí](https://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html). 

#### Objetivos:

 - Descargar y levantar los datos de Test. 
 - Separar los datos de las etiquetas.

In [ ]:
!pip install wget

In [13]:
import wget
#Descargo los datos de Test
wget.download('http://kdd.ics.uci.edu/databases/kddcup99/corrected.gz')

In [2]:
#Levanto los datos de Test
test_data = pd.read_csv('./corrected.gz', header=None)

test_data.columns = columns

#Se estandariza el formato de los datos en el Dataframe 
types = [float, str, str,str, float, float, str, float, float, float, float, str, float, float,float, float, float, float, float, float, str, str, 
         float, float, float, float,float, float, float, float,float, float, float, float, float, float, float,float, float, float, float, str]

for i in range(len(columns)):
    test_data[columns[i]] = test_data[columns[i]].astype(types[i])

test_data.head()

### Parte 2 - Preparar Conjuntos

Asumiendo que se mantuvo el orden en el que fueron descargados los datos:

#### Objetivos:

 - Separar los primeros 90000 datos de entrenamiento y sus respectivas etiquetas para el entrenamiento de los modelos. Estos deberían estar etiquetados como normales. 
 - Los restantes datos del conjunto de entrenamiento se utilizaran para validación.  Este conjunto permitirá fijar el punto de operación del modelo.
 - Cambie las etiquetas de los datos en todos los conjuntos a 0 y 1. Siendo 0 la etiqueta de la clase normal y 1 la etiqueta de la clase anómala.

### Parte 3 - Preprocesar y Visualizar los Datos

#### Objetivos:

 - Analizar y visualizar los datos. 
 - Realizar un **pipeline** que realice los los preprocesamientos que considere necesarios. 

# Detección de anomalías

Si bien *scikit-learn* proporciona algunas herramientas para la detección de anomalías, esta actividad se centra en la utilización de algoritmos no supervisados generalmente pensados para otras tareas como: reducción de la dimensionalidad, y/o *clustering*. Específicamente se trabajará con: PCA, *K-Means*, y *Gaussian Mixture Model* (GMM).

## PCA


### Parte 1


#### Objetivos:

 - Aplicar PCA sobre los datos de entrenamiento y graficar como varía el porcentaje de la varianza total en función de la cantidad de componentes principales (CPs). Se sugiere ver la sección *Choosing the Right Number of Dimensions* del capítulo 8 del libro. 
 - Determinar la cantidad de CPs de manera de mantener el *99%* de la varianza de los datos.
 - Utilizando PCA visualice los datos en dos o tres dimensiones. ¿Logra identificar clusters? *Se sugiere utilizar colores distintos para los puntos Normales y Anómales*

### Parte 2

La forma más directa de hacer detección de anomalías utilizando PCA, es mediante el error de reconstrucción. Para esto primero se calculan los componentes principales a partir de los datos reservados para el modelado. Luego, para cada dato a analizar se lo proyecta sobre estos componentes, y se calcula su reconstrucción. Debido a que los CPs fueron calculado sólo con datos normales, se espera que la reconstrucción de un dato anómalos tenga grandes errores. Es por esto que a partir del error de reconstrucción se pueda determinar si un dato es anómalo o no.

#### Objetivos:

*   Implementar un detector tal como se describe arriba, utilizando RMSE para calcular el error. El mismo se debe definir como una clase de manera que sea compatible con los *pipelines* de *scikit-learn*. En la siguiente celda se muestra un *template* para crear la clase ([aquí](https://scikit-learn.org/stable/developers/develop.html) se pueden ver otros ejemplos).

*   Crear un *pipeline* que incluya el preprocesamiento y el detector implementado.
*   Entrenar el modelo de manera que mantenga el *99%* de la varianza. 
*   Proponga un punto de operación teniendo en cuenta que se quiere evitar un exceso de falsas alarmas. Para ello se recomienda graficar el compromiso entre *precision* y *recall* para distintos valores de *threshold* que definen el punto de operación. Ver la sección *Precision/Recall Trade-off* del capítulo 3 del libro.
*   Graficar los *scores* de los datos utilizados en el punto anterior, diferenciando con colores los datos normales de los anómalos.
*    Evaluar el desempeño en el conjunto de Validación y Test. Puede resultar útil realizar una matriz de confusión. 

In [29]:
from sklearn.base import BaseEstimator, OutlierMixin
from sklearn.utils.validation import check_array, check_is_fitted
from sklearn.decomposition import PCA

class AD_PCA(BaseEstimator, OutlierMixin):
    def __init__(self, n_comp=None):
        self.n_comp = n_comp
    
    def fit(self, X, y=None):
        self.X = X
        self.y = y 
        self.PCA_ = PCA(n_components=self.n_comp)
        self.PCA_.fit(X)
        return self
    
    def score(self, X, y=None):
        X = check_array(X)
        check_is_fitted(self, ['X', 'y'])

        # Agregar código---

        #------------------
        return score

## K-Means

### Parte 1


#### Objetivos:

 - Proponer una forma de hallar la cantidad óptima de clusters. Se sugiere ver la sección *Finding the optimal number of clusters* del Capítulo 9. 

### Parte 2


Utilizando la cantidad óptima de cluster hallada:

#### Objetivos:

- Implementar un detector de anomalías utilizando *K-Means*. Para ello cree una clase y un *pipeline* que lo implemente de forma análoga a lo realizado con el método de PCA.
-   Proponga un punto de operación teniendo en cuenta el compromiso entre *precision* y *recall* para distintos valores de *threshold* que definen el punto de operación. 
-   Graficar los *scores* de los datos utilizados en el punto anterior, diferenciando con colores los datos normales de los anómalos. 
- Evaluar el desempeño en el conjunto de Validación y Test. Puede resultar útil realizar una matriz de confusión.

In [ ]:
from sklearn.base import BaseEstimator, OutlierMixin
from sklearn.cluster import KMeans
from sklearn.utils.validation import check_array, check_is_fitted

class AD_Kmeans(BaseEstimator, OutlierMixin):
    def __init__(self, n_clusters=8):
        self.K = n_clusters
    
    def fit(self, X, y=None):
        self.X = X
        self.y = y 
        self.Kmeans_ = KMeans(n_clusters=self.K)
        self.Kmeans_.fit(X)
        return self
    
    def score(self, X, y=None):
        X = check_array(X)
        check_is_fitted(self, ['X', 'y'])

         # Agregar código---

        #------------------
    
        return score

## Gaussian Mixtures Models

### Parte 1


Siguiendo el ejemplo de la sección *Anomaly Detection Using Gaussian Mixture* en el Capítulo 9 del libro. 

#### Objetivos:

 - Proponer una forma de hallar la cantidad óptima de mezclas a utilizar.



### Parte 2


#### Objetivos:

*   Implementar un detector que calcule el valor de los *scores*, y que además determine las predicciones a partir de un *threshold* calculado de manera similar al ejemplo del Capítulo 9. En la siguiente celda se proporciona un *template* para la implementación del detector.
*   Obtener diferentes predicciones para los datos del punto de operación,variando el parámetro *percen*. Discutir sobre los resultados.
- Evaluar el desempeño en el conjunto de Validación y Test. Puede resultar útil realizar una matriz de confusión.

In [80]:
from sklearn.mixture import GaussianMixture
from sklearn.base import BaseEstimator, OutlierMixin
from sklearn.utils.validation import check_array, check_is_fitted

In [51]:
class AD_GMM(BaseEstimator, OutlierMixin):

    def __init__(self, n_components=1, percen=0):
        self.n_comp = n_components
        self.percen = percen

    def fit(self, X, y=None):
        self.classes_ = [1, 0]
        self.X = X
        self.y = y 
        self.GMM_ = GaussianMixture(n_components=self.n_comp)
        self.GMM_.fit(X)

        #threshold---

        #------------
        return self
    
    def score(self, X, y=None):
        X = check_array(X)
        check_is_fitted(self, ['X', 'y'])

        # Agregar código---

        #------------------
        return score
    
    def predict(self, X):
        X = check_array(X)
        check_is_fitted(self, ['X', 'y'])

        # Agregar código---

        #------------------
        return pred 

# Opcional

*   Aplicar a los datos del problema alguno de los detectores de *sikit-learn* como: One-Class SVM, Isolation Forest
*   Comparar con los detectores anteriores.



# Sobre el Proyecto

## Punto de Operación

El desempeño del modelo desarrollado para resolver el problema de [detección del bosón de Higgs](https://www.kaggle.com/c/higgs-boson) se evaluará utilizando la métrica *AMS* que se define como: 

$$
\mathrm{AMS}=\sqrt{2\left(\left(s+b+b_{r}\right) \log \left(1+\frac{s}{b+b_{r}}\right)-s\right)}
$$

Donde: 

- $s$, $b$ : tasas de verdaderos positivos y falsos positivos no normalizados respectivamente.
- $b_r =10$ es el término de regularización. 
- $\log$ representa el logaritmo natural.

Se define el conjunto de datos $\mathcal{D}=\left\{\mathbf{x}_{\mathbf{i}}, y_{i}, w_{i}\right\}$ siendo $\mathbf{x}_{\mathbf{i}}$ las características físicas del experimento (medidas primitivas y derivadas), $y_{i} \in\{b, s\}$ el resultado del evento y $w_{i}$ el peso. Se denominan los subconjuntos de índices $\mathcal{S}=\left\{i: y_{i}=s\right\}$ y $\mathcal{B}=\left\{i: y_{i}=b\right\}$.

El cálculo de $s$ y $b$ se obtienen como: 

$$
\mathbf{s}=\sum_{i \in \mathcal{S} \cap \mathcal{G}} w_{i} \quad \mathbf{b}=\sum_{i \in \mathcal{B} \cap \mathcal{G}} w_{i}
$$

Donde se denota $\mathcal{G}=\left\{i: \hat{y}_{i}=s\right\}$, siendo $\hat{y}_{i}$ la predicción del modelo utilizado. Observar que $s$ y $b$ son las tasas de verdaderos y falsos positivos respectivamente como se mencionó anteriormente.

Descargue la implementación de la métrica de la página del [curso](https://eva.fing.edu.uy/mod/resource/view.php?id=135850). Modifique el nombre del archivo descargado a *HiggsBosonUtils.py* y guarde dicho archivo en una carpeta *tools* en el mismo directorio donde está el presente Notebook. En estas condiciones, puede importar la función *AMS* contenida dentro de *HiggsBosonUtils.py* de la siguiente forma:  

In [1]:
from tools.HiggsBosonUtils import AMS

In [3]:
s = 100
b = 1000000
AMS(s,b)

Al igual que como se trabajó en el presente taller, durante el proyecto será necesario también fijar un punto de operación que maximice el desempeño de la métrica, en este caso, el *AMS*. En función de ello, resulta pertinente observar que valores resultan razonables para dicha métrica. 

#### Objetivos:

 - Levantar el conjunto de datos del proyecto. 
 - Obtener el desempeño de *AMS* obtenido si el modelo clasifica todos los eventos como *background*.
 - Obtener el desempeño de *AMS* obtenido si el modelo clasifica todos los eventos como *signal*.
 - Obtener el desempeño de *AMS* obtenido si el modelo clasifica perfectamente todos los eventos.
 - Observe los Leaderboard de Kaggle y en base a los resultados obtenidos anteriormente que discuta que valores le resultan razonables de *AMS*.
 - Con los datos del proyecto genere dos conjuntos uno de Entrenamiento y otro de Validación. Luego, entrene el modelo y evalúe el desempeño en el conjunto de Validación. Observe como varía el AMS al modificar el umbral.